# Segmentace obrazu - počítání objektů
Cvičení je zaměřené na správné využití osvětlení při nasvícení objektu a následné využití metod pro segmentaci obrazu k počítání objektů. Je třeba se **zbavit odlesků**, **vyvážit bílou barvu** snímače, **vyladit jas** pomocí histogramu, atd...

<img src='images/lentilky_promo.jpg' style="width:50%">
<p style="text-align: center;">Ilustrační obrázek špatně nasnímaného objektu. :-)</p>

### Import knihoven a konfigurace

In [ ]:
%run ./library.ipynb

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu. Parametry a implementaci funkcí si můžete zobrazit pomocí příkazu `help(function_name)` nebo na https://gitlab.fit.cvut.cz/bi-svz/improutils_package/tree/master/improutils.

Seznam funkcí pro přehlednost:
- [`plot_images(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/visualisation/visualisation.py#L11)
- [`load_image(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/acquisition/img_io.py#L11)


- [`to_gray(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/preprocessing/preprocessing.py#L4)
- [`to_hsv(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/preprocessing/preprocessing.py#L21)
- [`negative(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/preprocessing/preprocessing.py#L53)
- [`crop(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/preprocessing/preprocessing.py#L99)


- [`segmentation_one_threshold(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/segmentation/segmentation.py#L56)
- [`segmentation_auto_threshold(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/segmentation/segmentation.py#L74)
- [`segmentation_two_thresholds(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/segmentation/segmentation.py#L90)
- [`find_contours(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/preprocessing/contours.py#L40)


- [`filtration_median(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/filtration/filtration.py#L22)


- [`to_intensity(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/segmentation/segmentation.py#L15)

---

### Úkol
Zvolte vhodné funkce pro segmentaci obrazu a spočítejte kolik se na obrázku vyskytuje celkem objektů. Využijte k tomu znalosti o segmentaci objektů podle barvy (*HINT: jiné barevné spektrum než RGB*) a postupně spočtěte objekty stejných barev, které nakonec sečtete dohromady.

Na obrázku je vidět převod barvy na úhel v jiném než RGB spektru.
<img src='images/color_wheel_hsv.png' style="width:25%">

Po vyladění algoritmu na jednotlivé úlohy, demonstrujte algoritmus na pořízeném snímku.

#### 1) Načtěte a zkontrolujte zaznamenaný obrázek.

In [ ]:
image_path = ... ###
image = ... ###
plot_images(image)

#### 2) Testovací/Pomocná funkce.
Zde si můžete interaktivně otestovat váš algoritmus.

In [ ]:
@interact( h_range=create_slider(min=0, max=360, description='Hue:'),
           s_range=create_slider(min=0, max=255, description='Saturation:'),
           v_range=create_slider(min=0, max=255, description='Value:'),)
def _(h_range,s_range,v_range):

    lower_bound=(to_intensity(h_range[0]), s_range[0], v_range[0]) ###
    upper_bound=(to_intensity(h_range[1]), s_range[1], v_range[1]) ###
    
    ###
    
    mask =  ... ###

    
    plot_images(mask, apply_mask(image, mask), titles=['Applied mask', 'Mask'])

#### 3) Doplňte algoritmus pro segmentaci objektů podle vybrané barvy. 
Vytvořte si konkrétní prahy, které budou vhodné pro danou barvu.

In [ ]:
def get_mask_for(img, color):
    if color=='red':
        ... ###
    elif color=='pink':
        ... ###
    elif color=='yellow':
        ... ###
    elif color=='orange':
        ... ###
    elif color=='purple':
        ... ###
    elif color=='green':
        ... ###
    elif color=='blue':
        ... ###
    else:
        raise ValueError("Invalid color selected.")

    ###
    return mask

In [ ]:
def algorithm(img, color):
    ### (Preprocessing (Filtrace etc.))
    ...
    
    ### Segmentace podle barvy
    mask_bin = get_mask_for(img_hsv, color)
    
    ###

    res = apply_mask(img, mask_bin)
    
    return res, mask_bin

#### 4) Doplňte funkci, která spočte objekty z binární masky.

In [ ]:
def count_objects(mask):
    min_area, max_area = ..., ... ###
    _, count, _ = ...(mask, min_area, max_area) ###
    return count

#### 5) Spusťte výsledný algoritmus pro spočtení objektů libovolné vybrané barvy v obraze. 

In [ ]:
selected_color = ... ###


result, mask = algorithm(image, selected_color)
count = count_objects(mask)

print(f'In total {count} {selected_color} objects found.')
plot_images(image, result)   